In [2]:
import os
import deflacue
import glob
import music_tag
import re

In [1]:
audio_ext = "flac" #assuming all files in flac
parent_dir = "/Users/u0126415/Downloads"

In [3]:
example_folder = "《鼓王》李小沛[WAV+CUE]"

In [17]:
cue_name = [x for x in os.listdir(f"{parent_dir}/{example_folder}") if x.endswith("cue") and not x.startswith("~")][0]

print(f"Cue file: {cue_name}")

cue_path = f"{parent_dir}/{example_folder}/{cue_name}"
output_dir = f"{parent_dir}/{example_folder}"

audio_names = [x for x in os.listdir(f"{parent_dir}/{example_folder}") if x.endswith(audio_ext)]
audio_names.sort()
print(f"Audio files: {audio_names}")

cover_file = [x for x in os.listdir(f"{parent_dir}/{example_folder}") if x.endswith("jpg")][0]
cover_path = f"{parent_dir}/{example_folder}/{cover_file}"
print(f"Cover files: {cover_file}")

####-----------####
print(f" ")


f = open(f"{parent_dir}/{example_folder}/{cue_name}", "r",encoding = "Chinese")
cue_str = f.read()
cue_str_seg = cue_str.split("\n")
tracks = [x for x in cue_str_seg if "TRACK" in x]
tracks_idx = [cue_str_seg.index(x) for x in tracks]

tracks_title_idx = [x+1 for x in tracks_idx]
tracks_artist_idx = [x+2 for x in tracks_idx]

#track_num = [x[8:-6] for x in tracks]  #please avoid hard indexing
track_num = [re.findall(r'\d+', x)[0] for x in tracks]

titles = [cue_str_seg[x].upper().split("TITLE")[1][2:-1] for x in tracks_title_idx]

if "PERFORMER" in cue_str:
    artists = [cue_str_seg[x].upper().split("PERFORMER")[1][2:-1] for x in tracks_artist_idx]
else:
    artists = [example_folder for x in range(len(artists))]
    
file_names = [f"{num}-{track}-{artist}" for (num,track,artist) in zip(track_num,titles,artists)]
file_names = [f"{x}.{audio_ext}" for x in file_names]

album = cue_str_seg[0][7:-1]
album = example_folder

print(album)
print(track_num)
print(titles)
print(artists)
print(file_names)

Cue file: CDImage.cue
Audio files: ['01 - р╧иНЁа.flac', '02 - я╪вс╟ХвЛ.flac', '03 - сФжш©╜╦Х.flac', '04 - о╥ё╗╣зр╩южубё╘.flac', '05 - юо╩╒д╔яю.flac', '06 - яТ╧ьхЩ╣Ч.flac', '07 - з╓оКжпм╗мЫн╟╢С╣дхЬЕхиЯцМж╝╡тцёнч╬║╣д╨Лмаб╥ио╣д╩ф╟ШЁ╞й╔уъцгаТтзо╕яТобдгЁак╪╣д╫ес║║╙н╙фТй╬иЯ╣дСПят║╒гЩ╦од╖╣д╨зс╟║╒уПЦ╓нрцг╣даИ╩Й║╒╪╓юЬнрцг╣дпдаИж╝иЯць║╒╨Й╢С║╒в╞яо║╒╦ъ╧С╨мвЁю╚╣д╧диЫ╪╟хЩ╟яЁ╛мя╨ЛЁ╬║╒спй╔м╫ж╝╬╚иЯ╣д╨Згы╤ЬвВ║ё.flac']
Cover files: cover.jpg
 
《鼓王》李小沛[WAV+CUE]
['01', '02', '03', '04', '05', '06', '07']
['夜深沉', '鸭子拌嘴', '渔舟凯歌', '戏（第一乐章）', '老虎磨牙', '阳关三叠', '冥想中通往伟大的萨迦神庙之苍茫无尽的红土路上的黄胞朝圣者们留在夕阳下那沉思的脚印―为启示神的箴言、驱赶魔的黑影、震悚我们的灵魂、激励我们的心灵之神秘、宏大、庄严、高贵和壮阔的鼓声及三把超脱红尘、有圣徒之精神的胡琴而作。']
['李小沛', '李小沛', '李小沛', '李小沛', '李小沛', '李小沛', '李小沛']
['01-夜深沉-李小沛.flac', '02-鸭子拌嘴-李小沛.flac', '03-渔舟凯歌-李小沛.flac', '04-戏（第一乐章）-李小沛.flac', '05-老虎磨牙-李小沛.flac', '06-阳关三叠-李小沛.flac', '07-冥想中通往伟大的萨迦神庙之苍茫无尽的红土路上的黄胞朝圣者们留在夕阳下那沉思的脚印―为启示神的箴言、驱赶魔的黑影、震悚我们的灵魂、激励我们的心灵之神秘、宏大、庄严、高贵和壮阔的鼓声及三把超脱红尘、有圣徒之精神的胡琴而作。-李小沛.flac']


In [18]:
#rename file and add tags
for file in audio_names:
    index_str = file[:2]

    if index_str in track_num:   #only track in cue list
        print(file)
        index = int(index_str)-1
        title = titles[index]
        artist = artists[index]

        file_name = file_names[index]
        try:
            os.rename(f"{parent_dir}/{example_folder}/{file}",f"{parent_dir}/{example_folder}/{file_name}")
            print(f"    {file} -> {file_name}")
        except:
            pass

        #-----------# Add tags
        f = music_tag.load_file(f"{parent_dir}/{example_folder}/{file_name}")

        f['title'] = titles[index]
        f['artist'] = artists[index]
        f['album'] = album
        
        if f['artwork'] == None:
            # set artwork
            with open(cover_path, 'rb') as img_in:
                f['artwork'] = img_in.read()
            with open(cover_path, 'rb') as img_in:
                f.append_tag('artwork', img_in.read())
            art = f['artwork']
            
            # Make a thumbnail
            art.first.thumbnail([64, 64])  # -> pillow image
            art.first.raw_thumbnail([64, 64])  # -> b'... raw thumbnail data ...'
        f.save()

    else:
        pass

01 - р╧иНЁа.flac
    01 - р╧иНЁа.flac -> 01-夜深沉-李小沛.flac
02 - я╪вс╟ХвЛ.flac
    02 - я╪вс╟ХвЛ.flac -> 02-鸭子拌嘴-李小沛.flac
03 - сФжш©╜╦Х.flac
    03 - сФжш©╜╦Х.flac -> 03-渔舟凯歌-李小沛.flac
04 - о╥ё╗╣зр╩южубё╘.flac
    04 - о╥ё╗╣зр╩южубё╘.flac -> 04-戏（第一乐章）-李小沛.flac
05 - юо╩╒д╔яю.flac
    05 - юо╩╒д╔яю.flac -> 05-老虎磨牙-李小沛.flac
06 - яТ╧ьхЩ╣Ч.flac
    06 - яТ╧ьхЩ╣Ч.flac -> 06-阳关三叠-李小沛.flac
07 - з╓оКжпм╗мЫн╟╢С╣дхЬЕхиЯцМж╝╡тцёнч╬║╣д╨Лмаб╥ио╣д╩ф╟ШЁ╞й╔уъцгаТтзо╕яТобдгЁак╪╣д╫ес║║╙н╙фТй╬иЯ╣дСПят║╒гЩ╦од╖╣д╨зс╟║╒уПЦ╓нрцг╣даИ╩Й║╒╪╓юЬнрцг╣дпдаИж╝иЯць║╒╨Й╢С║╒в╞яо║╒╦ъ╧С╨мвЁю╚╣д╧диЫ╪╟хЩ╟яЁ╛мя╨ЛЁ╬║╒спй╔м╫ж╝╬╚иЯ╣д╨Згы╤ЬвВ║ё.flac
    07 - з╓оКжпм╗мЫн╟╢С╣дхЬЕхиЯцМж╝╡тцёнч╬║╣д╨Лмаб╥ио╣д╩ф╟ШЁ╞й╔уъцгаТтзо╕яТобдгЁак╪╣д╫ес║║╙н╙фТй╬иЯ╣дСПят║╒гЩ╦од╖╣д╨зс╟║╒уПЦ╓нрцг╣даИ╩Й║╒╪╓юЬнрцг╣дпдаИж╝иЯць║╒╨Й╢С║╒в╞яо║╒╦ъ╧С╨мвЁю╚╣д╧диЫ╪╟хЩ╟яЁ╛мя╨ЛЁ╬║╒спй╔м╫ж╝╬╚иЯ╣д╨Згы╤ЬвВ║ё.flac -> 07-冥想中通往伟大的萨迦神庙之苍茫无尽的红土路上的黄胞朝圣者们留在夕阳下那沉思的脚印―为启示神的箴言、驱赶魔的黑影、震悚我们的灵魂、激励我们的心灵之神秘、宏大、庄严、高贵和壮阔的鼓声及三把超脱红尘、有圣徒之精神的胡琴而作。-李小沛.flac
